# IF3070 Foundations of Artificial Intelligence | Tugas Besar 2

Group Number: 30

Group Members:
- Lydia Gracia (18222035)
- Willhelmina Rachel Silalahi (18222049)
- Favian Izza Diasputra (18222070)
- Ervina Limka (18222100)

## Import Libraries

In [5]:
import pandas as pd
import numpy as np
from urllib.parse import urlparse  
import re
import difflib
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import joblib

## Import Dataset

In [6]:
# Example of reading a csv file from a gdrive link

# Take the file id from the gdrive file url
# https://drive.google.com/file/d/1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU/view?usp=sharing => The file id is 1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU
# and then put it in this format:
# https://drive.google.com/uc?id={file_id}
# Don't forget to change the access to public

# df = pd.read_csv('https://drive.google.com/uc?id=1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU')
# df.head()
sample = pd.read_csv('https://drive.google.com/uc?id=14PntjRvR-yo95ZSHj9B5yjrhz8EDWm12')
test = pd.read_csv('https://drive.google.com/uc?id=1m8yrgGVA-k97cPLA_Tzl0LIJg5AJbzWz')
train = pd.read_csv('https://drive.google.com/uc?id=1NjXOVHQ3db9DJUwkFiJ88M8xTZZhiamK')

display(train.head())
display(test.head())

,id,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,1,NaN,https://www.northcm.ac.th,24.0,www.northcm.ac.th,17.0,0.0,NaN,0.800000,NaN,...,0.0,0.0,1.0,NaN,3.0,NaN,69.0,NaN,NaN,1
1,4,8135291.txt,http://uqr.to/1il1z,NaN,NaN,NaN,NaN,to,1.000000,0.000896,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0
2,5,586561.txt,https://www.woolworthsrewards.com.au,35.0,www.woolworthsrewards.com.au,28.0,0.0,au,0.857143,NaN,...,1.0,0.0,1.0,33.0,7.0,8.0,15.0,NaN,2.0,1
3,6,NaN,NaN,31.0,NaN,NaN,NaN,com,0.562500,0.522907,...,1.0,0.0,1.0,24.0,5.0,14.0,NaN,NaN,NaN,1
4,11,412632.txt,NaN,NaN,www.nyprowrestling.com,22.0,0.0,NaN,1.000000,NaN,...,0.0,0.0,1.0,NaN,NaN,14.0,NaN,0.0,NaN,1


,"<!DOCTYPE html><html><head><title>Google Drive - Infected file</title><meta http-equiv=""content-type"" content=""text/html; charset=utf-8""/><style nonce=""Rh6WMfJ7XjEDHBKFML2yIQ"">.goog-link-button{position:relative;color:#15c;text-decoration:underline;cursor:pointer}.goog-link-button-disabled{color:#ccc;text-decoration:none;cursor:default}body{color:#222;font:normal 13px/1.4 arial",sans-serif;margin:0}.grecaptcha-badge{visibility:hidden}.uc-main{padding-top:50px;text-align:center}#uc-dl-icon{display:inline-block;margin-top:16px;padding-right:1em;vertical-align:top}#uc-text{display:inline-block;max-width:68ex;text-align:left}.uc-error-caption,".uc-warning-caption{color:#222;font-size:16px}#uc-download-link{text-decoration:none}.uc-name-size a{color:#15c;text-decoration:none}.uc-name-size a:visited{color:#61c;text-decoration:none}.uc-name-size a:active{color:#d14836;text-decoration:none}.uc-footer{color:#777;font-size:11px;padding-bottom:5ex;padding-top:5ex;text-align:center}.uc-footer a{color:#15c}.uc-footer a:visited{color:#61c}.uc-footer a:active{color:#d14836}.uc-footer-divider{color:#ccc;width:100%}.goog-inline-block{position:relative;display:-moz-inline-box;display:inline-block}* html .goog-inline-block{display:inline}*:first-child+html .goog-inline-block{display:inline}sentinel{}</style><link rel=""icon"" href=""//ssl.gstatic.com/docs/doclist/images/drive_2022q3_32dp.png""/></head><body><div class=""uc-main""><div id=""uc-text""><p class=""uc-error-caption"">Sorry","this file is infected with a virus.</p><p class=""uc-error-subcaption"">Only the owner is allowed to download infected files.</p></div></div><div class=""uc-footer""><hr class=""uc-footer-divider""></div></body></html>"


# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [7]:
# Split training set and validation set here, store into variables train_set and val_set.
# Remember to also keep the original training set before splitting. This will come important later.
# train_set, val_set = ...

# Split the data
train_set, val_set = train_test_split(
    train,
    test_size=0.2,              # 20% for validation
    stratify=train['label'],    # Ensure the same proportion of '0' and '1'
    random_state=42
)

In [8]:
# Display the shapes to verify
print("Original dataset shape:", train.shape)
print("Training set shape:", train_set.shape)
print("Validation set shape:", val_set.shape)
print("")

# Verify the distribution of the target variable
print("Label distribution in training set:\n", train_set['label'].value_counts(normalize=True))
print("")
print("Label distribution in validation set:\n", val_set['label'].value_counts(normalize=True))
print("")

Original dataset shape: (140404, 56)
Training set shape: (112323, 56)
Validation set shape: (28081, 56)

Label distribution in training set:
 1    0.924833
0    0.075167
Name: label, dtype: float64

Label distribution in validation set:
 1    0.924825
0    0.075175
Name: label, dtype: float64



# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

#### Cell Gracie

In [9]:
# Check for missing values and calculate their percentages
def missing_data_summary (df):
    missing_values = df.isnull().sum()
    missing_percentage = (missing_values / len(df)) * 100

    # Combine into a DataFrame for better visualization
    missing = pd.DataFrame({
        'Feature': df.columns,
        'Missing Values': missing_values,
        'Percentage (%)': missing_percentage
    }).sort_values(by='Percentage (%)', ascending=False)

    return display(missing)

# Filll missing URL with IsHTTPS and Domain
def fill_missing_url(df):
    mask = df['URL'].isna() & df['IsHTTPS'].notna() & df['Domain'].notna()
    affected = df.loc[mask, ['id', 'IsHTTPS', 'Domain', 'URL', 'URLLength']].copy()
    affected['LengthDiff'] = affected.apply(lambda row: row['URLLength'] - len(('https' if row['IsHTTPS'] == 1 else 'http') + '://' + row['Domain']) if not np.isnan(row['URLLength']) else np.nan, axis=1)

    df.loc[mask, 'URL'] = df.loc[mask].apply(lambda row: ('https' if row['IsHTTPS'] == 1 else 'http') + '://' + row['Domain'], axis=1)
    affected['URL'] = df.loc[mask, 'URL']

    display(affected)
    print("Affected rows with fitting URLLength:", affected['LengthDiff'].eq(0).sum())
    print("Affected rows with non-fitting URLLength:", affected['LengthDiff'].dropna().ne(0).sum())
    print("Affected rows with NaN URLLength:", affected['LengthDiff'].isna().sum())
    
    return df, affected

# Fill missing URL with Domain
def fill_missing_url_domain(df):
    mask = df['URL'].isna() & df['Domain'].notna()
    affected = df.loc[mask, ['id', 'Domain', 'URL', 'URLLength']].copy()
    affected['LengthDiff'] = affected.apply(lambda row: row['URLLength'] - len(row['Domain']) if not np.isnan(row['URLLength']) else np.nan, axis=1)

    df.loc[mask, 'URL'] = df.loc[mask].apply(lambda row: row['Domain'], axis=1)
    affected['URL'] = df.loc[mask, 'URL']

    display(affected)   
    print("Affected rows with fitting URLLength:", affected['LengthDiff'].eq(0).sum())
    print("Affected rows with non-fitting URLLength:", affected['LengthDiff'].dropna().ne(0).sum())
    print("Affected rows with NaN URLLength:", affected['LengthDiff'].isna().sum())

    return df, affected

# Fill missing URLLength with URL
def fill_missing_urllength(df):
    mask = df['URLLength'].isna() & df['URL'].notna()
    affected = df.loc[mask, ['id', 'URL', 'URLLength']].copy()

    df.loc[mask, 'URLLength'] = df.loc[mask, 'URL'].apply(lambda x: len(x))
    affected['URLLength'] = df.loc[mask, 'URLLength']

    display(affected)

    return df

# Fill missing Domain with URL
def fill_missing_domain(df):
    mask = df['Domain'].isna() & df['URL'].notna()
    affected = df.loc[mask, ['id', 'URL', 'Domain']].copy()

    df.loc[mask, 'Domain'] = df.loc[mask, 'URL'].apply(
        lambda x: x.split('/')[0] + '//' + x.split('://', 1)[-1].split('/')[0] if '://' in x else x.split('/')[0]
    )
    affected['Domain'] = df.loc[mask, 'Domain']

    display(affected)

    return df

# Fill misisng TLD values with Domain
def fill_missing_tld(df):
    mask = df['TLD'].isna() & df['Domain'].notna()
    affected = df.loc[mask, ['id', 'Domain']].copy()
    affected['TLD'] = affected['Domain'].apply(lambda domain: domain.split('.')[-1] if len(domain.split('.')) > 1 else np.nan)

    df.loc[mask, 'TLD'] = affected['TLD']

    display(affected)

    return df

# Fill missing IsDomainIP with Domain
def fill_missing_isdomainip(df):
    mask = df['IsDomainIP'].isna() & df['Domain'].notna()
    affected = df.loc[mask, ['id', 'Domain', 'IsDomainIP']].copy()

    df.loc[mask, 'IsDomainIP'] = df.loc[mask, 'Domain'].apply(lambda x: int(x.replace('.', '').isdigit()))
    affected['IsDomainIP'] = df.loc[mask, 'IsDomainIP']

    display(affected)

    return df

# Fill missing DomainLength with Domain
def fill_missing_domainlength(df):
    mask = df['DomainLength'].isna() & df['Domain'].notna()
    affected = df.loc[mask, ['id', 'Domain', 'DomainLength']].copy()

    df.loc[mask, 'DomainLength'] = df.loc[mask, 'Domain'].apply(lambda x: len(x))
    affected['DomainLength'] = df.loc[mask, 'DomainLength']

    display(affected)

    return df


In [10]:
missing_data_summary(train_set)

,Feature,Missing Values,Percentage (%)
Domain,Domain,56122,49.964834
NoOfExternalRef,NoOfExternalRef,55549,49.454698
LineOfCode,LineOfCode,55320,49.250821
LargestLineLength,LargestLineLength,54374,48.408607
HasSocialNet,HasSocialNet,54363,48.398814
NoOfURLRedirect,NoOfURLRedirect,53925,48.008867
HasCopyrightInfo,HasCopyrightInfo,53879,47.967914
NoOfCSS,NoOfCSS,53646,47.760476
NoOfObfuscatedChar,NoOfObfuscatedChar,53443,47.579748
NoOfSelfRedirect,NoOfSelfRedirect,53422,47.561052


In [11]:
# Drop FILENAME
train_set = train.drop(columns='FILENAME')

In [12]:
# Fill missing URL with IsHTTPS and Domain
train_set, affected_URL = fill_missing_url(train_set)

missing_data_summary(train_set[['URL']])

,id,IsHTTPS,Domain,URL,URLLength,LengthDiff
4,11,1.0,www.nyprowrestling.com,https://www.nyprowrestling.com,NaN,NaN
14,32,1.0,www.scientistsforeu.uk,https://www.scientistsforeu.uk,29.0,-1.0
21,44,1.0,www.tourbytransit.com,https://www.tourbytransit.com,28.0,-1.0
27,55,1.0,www.speaker.gov,https://www.speaker.gov,NaN,NaN
28,57,1.0,storageapi.fleek.one,https://storageapi.fleek.one,NaN,NaN
...,...,...,...,...,...,...
140369,235741,1.0,www.spellmanbrady.com,https://www.spellmanbrady.com,28.0,-1.0
140378,235753,1.0,www.hildegardwesterkamp.ca,https://www.hildegardwesterkamp.ca,NaN,NaN
140388,235771,1.0,www.clan-cameron.org.au,https://www.clan-cameron.org.au,NaN,NaN
140393,235780,1.0,www.fites.net,https://www.fites.net,NaN,NaN


Affected rows with fitting URLLength: 144
Affected rows with non-fitting URLLength: 7900
Affected rows with NaN URLLength: 6117


,Feature,Missing Values,Percentage (%)
URL,URL,29326,20.886869


#### Analyze URL imputation from IsHTTPS and Domain

In [13]:
affected_URL['LengthDiff'].dropna().ne(0)
affected_URL = affected_URL[affected_URL['LengthDiff'].notna() & affected_URL['LengthDiff'].ne(0)]
affected_URL.sort_values(by='LengthDiff', inplace=True)
display(affected_URL)

,id,IsHTTPS,Domain,URL,URLLength,LengthDiff
14,32,1.0,www.scientistsforeu.uk,https://www.scientistsforeu.uk,29.0,-1.0
92589,155254,1.0,www.gifex.com,https://www.gifex.com,20.0,-1.0
92561,155211,1.0,www.translatearticles.com,https://www.translatearticles.com,32.0,-1.0
92557,155195,1.0,www.kent.edu,https://www.kent.edu,19.0,-1.0
92533,155153,1.0,www.power-flowers.com,https://www.power-flowers.com,28.0,-1.0
...,...,...,...,...,...,...
91023,152612,1.0,43.134.163.95,https://43.134.163.95,187.0,166.0
66629,111404,0.0,www.kuerennkaycoato-co-jp.kuercnokayccata.zxvt...,http://www.kuerennkaycoato-co-jp.kuercnokaycca...,251.0,192.0
110603,185718,1.0,fingerleaf.fr,https://fingerleaf.fr,242.0,221.0
133923,224822,1.0,stage.microsoft.safereset.co,https://stage.microsoft.safereset.co,320.0,284.0


In [14]:
display(affected_URL['LengthDiff'].describe())

count    7900.000000
mean       -0.065063
std         9.718265
min        -1.000000
25%        -1.000000
50%        -1.000000
75%        -1.000000
max       396.000000
Name: LengthDiff, dtype: float64

In [15]:
# Count the number of affected_URL rows with LengthDiff equal to 1
print("Number of rows with LengthDiff equal to 1:", affected_URL[affected_URL['LengthDiff'] == 1].shape[0])

# Display affected_URL with extreme LengthDiff
display(affected_URL[(affected_URL['LengthDiff'] != 1) & (affected_URL['LengthDiff'] != -1)])

Number of rows with LengthDiff equal to 1: 117


,id,IsHTTPS,Domain,URL,URLLength,LengthDiff
39516,66083,1.0,www.gpstrainingform.com,https://www.gpstrainingform.com,34.0,3.0
105549,177137,1.0,rcuorgreview.rhwc.com.au,https://rcuorgreview.rhwc.com.au,35.0,3.0
55500,92781,1.0,ijejfddq.server1.trinchera.dev,https://ijejfddq.server1.trinchera.dev,42.0,4.0
86997,145846,0.0,www.kueronekayacotn-co-jp.kuerocekayacoto.gbyk...,http://www.kueronekayacotn-co-jp.kuerocekayaco...,63.0,4.0
65796,109983,1.0,s.id,https://s.id,17.0,5.0
...,...,...,...,...,...,...
91023,152612,1.0,43.134.163.95,https://43.134.163.95,187.0,166.0
66629,111404,0.0,www.kuerennkaycoato-co-jp.kuercnokayccata.zxvt...,http://www.kuerennkaycoato-co-jp.kuercnokaycca...,251.0,192.0
110603,185718,1.0,fingerleaf.fr,https://fingerleaf.fr,242.0,221.0
133923,224822,1.0,stage.microsoft.safereset.co,https://stage.microsoft.safereset.co,320.0,284.0


#### Analyze given URL and URLLength match

In [16]:
# Select rows in train_set that id is not in affected_URL and doesn't have Nan URL nor URLLength
train_def = train_set[~train_set['id'].isin(affected_URL['id'])]
train_def = train_def.dropna(subset=['URL', 'URLLength'])

# Calculate LengthDiff of URL.len and URLLength
train_def['LengthDiff'] = train_def.apply(lambda row: len(row['URL']) - int(row['URLLength']), axis=1)

# Display the rows with non-zero LengthDiff
display(train_def[train_def['LengthDiff'] != 0][['id', 'URL', 'URLLength', 'LengthDiff']])

# Display the rows with zero LengthDiff
display(train_def[train_def['LengthDiff'] == 0][['id', 'URL', 'URLLength', 'LengthDiff']])

,id,URL,URLLength,LengthDiff
0,1,https://www.northcm.ac.th,24.0,1
2,5,https://www.woolworthsrewards.com.au,35.0,1
5,12,https://www.free-marine.com,26.0,1
6,17,https://www.ridemcts.com,23.0,1
9,22,https://www.screenbeam.com,25.0,1
...,...,...,...,...
140397,235786,https://www.nnry.com,19.0,1
140400,235791,https://www.greenmountainenergy.com,34.0,1
140401,235792,https://www.leadcastingcall.com,30.0,1
140402,235794,https://www.fedarb.com,21.0,1


,id,URL,URLLength,LengthDiff
105,179,https://thebabytheater.com/00ad7e3/,35.0,0
235,392,https://onlineauth-1c.cfd/43478,31.0,0
272,446,https://track-il.web.app/,25.0,0
311,505,https://bradesco-livelo-ocnqpjugja-ew.a.run.app/,48.0,0
464,761,https://aol-106703.weeblysite.com/,34.0,0
...,...,...,...,...
140018,235166,https://sm3gecmtm.web.app/,26.0,0
140028,235179,http://app-davis.aahjuwfofx-xlm419p0x6dy.p.tem...,58.0,0
140329,235667,http://kraken-loiggiinc.godaddysites.com,40.0,0
140341,235685,http://www.lastorb.com,22.0,0


**End of analysis**

In [17]:
# Fill missing URL with Domain
train_set, affected_URL = fill_missing_url_domain(train_set)

missing_data_summary(train_set[['URL']])

,id,Domain,URL,URLLength,LengthDiff
54,105,www.andy-ngo.com,www.andy-ngo.com,NaN,NaN
76,136,verify-facebook.qcom.biz.id,verify-facebook.qcom.biz.id,NaN,NaN
111,186,www.moas.eu,www.moas.eu,NaN,NaN
125,214,www.snhu.edu,www.snhu.edu,19.0,7.0
141,240,www.equationsdigital.com,www.equationsdigital.com,NaN,NaN
...,...,...,...,...,...
140276,235568,www.kate.io,www.kate.io,18.0,7.0
140290,235593,ipfs.io,ipfs.io,66.0,59.0
140310,235635,www.damianieditore.com,www.damianieditore.com,NaN,NaN
140322,235657,www.pymssql.org,www.pymssql.org,NaN,NaN


Affected rows with fitting URLLength: 0
Affected rows with non-fitting URLLength: 4323
Affected rows with NaN URLLength: 3243


,Feature,Missing Values,Percentage (%)
URL,URL,21760,15.498134


##### Analyze URL imputation from Domain

In [18]:
display(affected_URL['LengthDiff'].describe())

count    4323.000000
mean        8.181818
std        10.334313
min         6.000000
25%         7.000000
50%         7.000000
75%         7.000000
max       241.000000
Name: LengthDiff, dtype: float64

**End of analysis**

In [19]:
# Fill missing URLLength with URL
train_set = fill_missing_urllength(train_set)

missing_data_summary(train_set[['URLLength']])

,id,URL,URLLength
1,4,http://uqr.to/1il1z,19.0
4,11,https://www.nyprowrestling.com,30.0
7,19,https://www.epner.com,21.0
8,20,https://www.az511.com,21.0
10,23,https://www.hellscent.com,25.0
...,...,...,...
140388,235771,https://www.clan-cameron.org.au,31.0
140393,235780,https://www.fites.net,21.0
140394,235782,https://www.bmvc2020-conference.com,35.0
140396,235785,https://www.genevalakemuseum.org,32.0


,Feature,Missing Values,Percentage (%)
URLLength,URLLength,9425,6.712772


In [20]:
# Fill missing Domain with URL
train_set = fill_missing_domain(train_set)

missing_data_summary(train_set[['Domain']])

,id,URL,Domain
1,4,http://uqr.to/1il1z,http://uqr.to
5,12,https://www.free-marine.com,https://www.free-marine.com
7,19,https://www.epner.com,https://www.epner.com
10,23,https://www.hellscent.com,https://www.hellscent.com
13,29,https://www.uncrazed.com,https://www.uncrazed.com
...,...,...,...
140387,235769,https://www.mdx.edu.mt,https://www.mdx.edu.mt
140389,235774,https://www.navigazionelaghi.it,https://www.navigazionelaghi.it
140395,235784,https://www.inspiredherway.com,https://www.inspiredherway.com
140397,235786,https://www.nnry.com,https://www.nnry.com


,Feature,Missing Values,Percentage (%)
Domain,Domain,21760,15.498134


In [21]:
# Fill missing TLD with Domain
train_set = fill_missing_tld(train_set)

display(missing_data_summary(train_set[['Domain']]))

,id,Domain,TLD
0,1,www.northcm.ac.th,th
4,11,www.nyprowrestling.com,com
9,22,www.screenbeam.com,com
11,25,www.marcosimoncellifondazione.it,it
12,27,www.marxist.se,se
...,...,...,...
140397,235786,https://www.nnry.com,com
140399,235790,www.slavevoyages.org,org
140401,235792,www.leadcastingcall.com,com
140402,235794,https://www.fedarb.com,com


,Feature,Missing Values,Percentage (%)
Domain,Domain,21760,15.498134


None

In [22]:
# Fill missing IsDomainIP with Domain
train_set = fill_missing_isdomainip(train_set)

missing_data_summary(train_set[['IsDomainIP']])

,id,Domain,IsDomainIP
1,4,http://uqr.to,0.0
5,12,https://www.free-marine.com,0.0
9,22,www.screenbeam.com,0.0
10,23,https://www.hellscent.com,0.0
14,32,www.scientistsforeu.uk,0.0
...,...,...,...
140389,235774,https://www.navigazionelaghi.it,0.0
140395,235784,https://www.inspiredherway.com,0.0
140401,235792,www.leadcastingcall.com,0.0
140402,235794,https://www.fedarb.com,0.0


,Feature,Missing Values,Percentage (%)
IsDomainIP,IsDomainIP,6520,4.643742


In [23]:
# Fill missing DomainLength with Domain
train_set = fill_missing_domainlength(train_set)

missing_data_summary(train_set[['DomainLength']])

,id,Domain,DomainLength
1,4,http://uqr.to,13.0
7,19,https://www.epner.com,21.0
15,37,www.goldreserveinc.com,22.0
17,39,www.topografix.com,18.0
19,42,www.saveware.nl,15.0
...,...,...,...
140387,235769,https://www.mdx.edu.mt,22.0
140394,235782,www.bmvc2020-conference.com,27.0
140397,235786,https://www.nnry.com,20.0
140399,235790,www.slavevoyages.org,20.0


,Feature,Missing Values,Percentage (%)
DomainLength,DomainLength,7223,5.14444


In [24]:
missing_data_summary(train_set)

,Feature,Missing Values,Percentage (%)
NoOfExternalRef,NoOfExternalRef,69379,49.413834
LineOfCode,LineOfCode,69153,49.252870
HasSocialNet,HasSocialNet,67999,48.430956
LargestLineLength,LargestLineLength,67928,48.380388
NoOfURLRedirect,NoOfURLRedirect,67384,47.992935
HasCopyrightInfo,HasCopyrightInfo,67345,47.965158
NoOfCSS,NoOfCSS,67134,47.814877
NoOfObfuscatedChar,NoOfObfuscatedChar,66798,47.575568
NoOfSelfRedirect,NoOfSelfRedirect,66715,47.516453
HasPasswordField,HasPasswordField,66535,47.388251


#### Cell Favian

In [25]:
def fill_tld_length(df):
    df['TLDLength'] = df['TLD'].apply(lambda x: len(x) if pd.notnull(x) else None)
    return df

def get_obfuscated_chars(url):
    if pd.isnull(url):
        return None
    obfuscated_chars = []
    common_patterns = [r'%[0-9a-fA-F]{2}', r'\\x[0-9a-fA-F]{2}', r'\\u[0-9a-fA-F]{4}']
    for pattern in common_patterns:
        obfuscated_chars.extend(re.findall(pattern, url))
    return len(obfuscated_chars)

def fill_no_of_obfuscated_char(df):
    df['NoOfObfuscatedChar'] = df.apply(
        lambda row: (
            get_obfuscated_chars(row['URL'])
            if pd.notnull(row['URL']) and (
                pd.isnull(row['HasObfuscation']) or pd.isnull(row['ObfuscationRatio'])
            )
            else round(row['ObfuscationRatio'] * row['URLLength'])
            if pd.notnull(row['ObfuscationRatio']) and pd.notnull(row['URLLength'])
            else None
        ),
        axis=1,
    )
    return df

def fill_has_obfuscation(df):
    df['HasObfuscation'] = df['NoOfObfuscatedChar'].apply(
        lambda x: 1 if pd.notnull(x) and x > 0 else None
    )
    return df

def fill_no_of_subdomain(df):
    df['NoOfSubDomain'] = df['NoOfSubDomain'].where(
        ~df['NoOfSubDomain'].isnull(),
        df['Domain'].apply(lambda x: x.count('.') - 1 if pd.notnull(x) else None),
    )
    return df

def fill_no_of_letters_in_url(df):
    def count_letters_excluding_prefix(url):
        if pd.isnull(url):
            return None
        url_without_prefix = re.sub(r'^(https?://)?(www\.)?', '', url)
        return len(re.findall(r'[a-zA-Z]', url_without_prefix))

    df['NoOfLettersInURL'] = df['NoOfLettersInURL'].where(
        ~df['NoOfLettersInURL'].isnull(),
        df['URL'].apply(count_letters_excluding_prefix),
    )
    return df

def fill_no_of_digits_in_url(df):
    def count_digits_excluding_prefix(url):
        if pd.isnull(url):
            return None
        url_without_prefix = re.sub(r'^(https?://)?(www\.)?', '', url)
        return len(re.findall(r'[0-9]', url_without_prefix))

    df['NoOfDegitsInURL'] = df['NoOfDegitsInURL'].where(
        ~df['NoOfDegitsInURL'].isnull(),
        df['URL'].apply(count_digits_excluding_prefix),
    )
    return df

def fill_no_of_equals_in_url(df):
    df['NoOfEqualsInURL'] = df['URL'].apply(
        lambda x: x.count('=') if pd.notnull(x) else None
    )
    return df

def fill_no_of_qmark_in_url(df):
    def count_qmarks(url):
        if pd.isnull(url):
            return None
        url_without_prefix = re.sub(r'^(https?://)?(www\.)?', '', url)
        return url_without_prefix.count('?')

    df['NoOfQMarkInURL'] = df['NoOfQMarkInURL'].where(
        ~df['NoOfQMarkInURL'].isnull(),
        df['URL'].apply(count_qmarks),
    )
    return df

def fill_no_of_ampersand_in_url(df):
    df['NoOfAmpersandInURL'] = df['URL'].apply(
        lambda x: x.count('&') if pd.notnull(x) else None
    )
    return df

def fill_no_of_other_special_chars_in_url(df):
    def count_other_special_chars(url):
        if pd.isnull(url):
            return None
        url_without_prefix = re.sub(r'^(https?://)?(www\.)?', '', url)
        return len(re.findall(r'[^a-zA-Z0-9&?=]', url_without_prefix))

    df['NoOfOtherSpecialCharsInURL'] = df['NoOfOtherSpecialCharsInURL'].where(
        ~df['NoOfOtherSpecialCharsInURL'].isnull(),
        df['URL'].apply(count_other_special_chars),
    )
    return df

def fill_features_from_url(df):
    df = fill_tld_length(df)
    df = fill_no_of_obfuscated_char(df)
    df = fill_has_obfuscation(df)
    df = fill_no_of_subdomain(df)
    df = fill_no_of_letters_in_url(df)
    df = fill_no_of_digits_in_url(df)
    df = fill_no_of_equals_in_url(df)
    df = fill_no_of_qmark_in_url(df)
    df = fill_no_of_ampersand_in_url(df)
    df = fill_no_of_other_special_chars_in_url(df)
    return df

In [26]:
def fill_missing_values(df):
    def safe_mode(series):
        if series.dropna().empty:  
            return 0  
        return series.mode()[0]

    def predict_based_on_correlation(row, correlated_features, threshold=0.5):
        total_score = 0
        count = 0
        for feature in correlated_features:
            if pd.notnull(row[feature]):
                total_score += row[feature]
                count += 1
        if count == 0:  
            return safe_mode(pd.Series(correlated_features))
        return 1 if (total_score / count) >= threshold else 0

    # Fill missing values for each column
    
    # < 0.3 correlation with label
    df['NoOfURLRedirect'] = df['NoOfURLRedirect'].fillna(df['NoOfURLRedirect'].median())
    df['NoOfSelfRedirect'] = df['NoOfSelfRedirect'].fillna(0)
    df['NoOfPopup'] = df['NoOfPopup'].fillna(0)
    df['NoOfiFrame'] = df['NoOfiFrame'].fillna(0)
    df['HasExternalFormSubmit'] = df['HasExternalFormSubmit'].fillna(safe_mode(df['HasExternalFormSubmit']))
    df['HasPasswordField'] = df['HasPasswordField'].fillna(safe_mode(df['HasPasswordField']))
    df['Bank'] = df['Bank'].fillna(0)
    df['Crypto'] = df['Crypto'].fillna(0)
    df['NoOfImage'] = df['NoOfImage'].fillna(df['NoOfImage'].median())
    df['NoOfCSS'] = df['NoOfCSS'].fillna(df['NoOfCSS'].median())
    df['NoOfEmptyRef'] = df['NoOfEmptyRef'].fillna(0)
    df['NoOfExternalRef'] = df['NoOfExternalRef'].fillna(df['NoOfExternalRef'].median())

    # < 0.5 correlation with label
    df['Robots'] = df['Robots'].fillna(safe_mode(df['Robots']))
    df['Pay'] = df['Pay'].fillna(safe_mode(df['Pay']))
    df['NoOfJS'] = df['NoOfJS'].fillna(df['NoOfJS'].median())
    df['NoOfSelfRef'] = df['NoOfSelfRef'].fillna(0)
    df['HasFavicon'] = df['HasFavicon'].fillna(safe_mode(df['HasFavicon'])) 

    # >= 0.45 correlation with label, use correlated variables for imputation
    df['HasSocialNet'] = df['HasSocialNet'].fillna(safe_mode(df['HasSocialNet']))
    df['HasHiddenFields'] = df['HasHiddenFields'].fillna(safe_mode(df['HasHiddenFields']))
    df['HasSubmitButton'] = df['HasSubmitButton'].fillna(safe_mode(df['HasSubmitButton']))
    df['HasCopyrightInfo'] = df['HasCopyrightInfo'].fillna(safe_mode(df['HasCopyrightInfo']))
    df['HasDescription'] = df['HasDescription'].fillna(safe_mode(df['HasDescription']))
    df['IsResponsive'] = df['IsResponsive'].fillna(safe_mode(df['IsResponsive'])) 
    
    return df

In [27]:
train_set = fill_features_from_url(train_set)
train_set = fill_missing_values(train_set)

#### Cell Ervina

In [29]:
def fill_missing_isHTTPS(df):
    df['IsHTTPS'] = df['URL'].apply(lambda x: 1 if isinstance(x, str) and x.startswith('https') else 0)
    display(df[['URL', 'IsHTTPS']])
    return df

def fill_missing_hasTitle(df):
    df['HasTitle'] = df.apply(
    lambda row: 0 if pd.isna(row['HasTitle']) and pd.isna(row['Title'])
    else (1 if pd.isna(row['HasTitle']) and not pd.isna(row['Title']) else row['HasTitle']), axis=1)
    display(df[['Title', 'HasTitle']])
    return df

def fill_missing_title(df):
    def extract_domain_name(domain):
        if pd.isna(domain):
            return np.nan
        domain = domain.replace('http://', '').replace('https://', '').replace('www.', '')
        return domain.split('.')[0] 

    def fill_title(row):
        if pd.isna(row['Title']) and row['DomainTitleMatchScore'] == 100.0:
            return extract_domain_name(row['Domain'])
        elif row['DomainTitleMatchScore'] == 0.0:
            return row['Title']
        else:
            return row['Title'] 
    
    df['Title'] = df.apply(fill_title, axis=1)
    display(df[['Domain', 'Title', 'DomainTitleMatchScore']])
    return df

def fill_missing_domaintitle(df):
    def extract_domain(domain):
        if pd.isna(domain) or domain == '':
            return ''
        
        try:
            parsed_domain = urlparse(domain)
            domain = parsed_domain.netloc.replace('www.', '')
            
            if not domain and parsed_domain.path:
                domain = parsed_domain.path.strip('/').split('/')[0]
            
            domain = re.sub(r'\.(com|org|net|edu|gov|io|co|info)$', '', domain)
            
            return domain.lower()
        except Exception as e:
            print(f"Error extracting domain: {domain}, Error: {e}")
            return ''

    def clean_text(text):
        if pd.isna(text):
            return ''
        
        text = str(text).lower()
        text = re.sub(r'[^a-z0-9]', '', text)
        
        return text

    def advanced_domain_title_match(domain, title):
        cleaned_domain = extract_domain(domain)
        cleaned_title = clean_text(title)
        
        if not cleaned_domain or not cleaned_title:
            return np.nan
        
        # Define multiple strategies for matching domain and title
        strategies = [
            (lambda d, t: 100 if d in t else 0, 0.4),  # Exact match (40% weight)
            (lambda d, t: difflib.SequenceMatcher(None, d, t).ratio() * 100, 0.3),  # Similarity ratio (30%)
            (lambda d, t: len(set(d) & set(t)) / len(set(d+t)) * 100 if d and t else 0, 0.2),  # Shared characters (20%)
            (lambda d, t: 50 if any(char in t for char in d) else 0, 0.1)  # Partial character match (10%)
        ]
        
        total_score = 0
        for strategy, weight in strategies:
            try:
                score = strategy(cleaned_domain, cleaned_title)
                total_score += score * weight
            except Exception as e:
                print(f"Error in strategy: {e}")
        
        return min(max(total_score, 0), 100)

    def handle_domain_title_match_score(df):
        df_copy = df.copy()
        
        # Mask rows where DomainTitleMatchScore is missing
        mask_missing = df_copy['DomainTitleMatchScore'].isna()
    
        # Safe calculation of match score
        def safe_match_calculation(row):
            try:
                score = advanced_domain_title_match(row['Domain'], row['Title'])
                return score if not np.isnan(score) else 0
            except Exception as e:
                print(f"Error calculating match: {e}")
                return 0
        
        # Update only rows with missing DomainTitleMatchScore
        df_copy.loc[mask_missing, 'DomainTitleMatchScore'] = df_copy.loc[mask_missing].apply(
            safe_match_calculation, 
            axis=1
        )

        return df_copy

    # Process the DataFrame
    df_processed = handle_domain_title_match_score(df)
    display(df_processed[['Domain', 'Title', 'DomainTitleMatchScore']])
    
    return df_processed

def fill_missing_urltitle(df):
    def extract_url(url):
        if pd.isna(url) or url == '':
            return ''
        
        try:
            parsed_url = urlparse(url)
            url = parsed_url.netloc.replace('www.', '')
            
            if not url and parsed_url.path:
                url = parsed_url.path.strip('/').split('/')[0]
            
            url = re.sub(r'\.(com|org|net|edu|gov|io|co|info)$', '', url)
            
            return url.lower()
        except Exception as e:
            print(f"Error extracting URL: {url}, Error: {e}")
            return ''

    def clean_text(text):
        if pd.isna(text):
            return ''
        
        text = str(text).lower()
        
        text = re.sub(r'[^a-z0-9]', '', text)
        
        return text

    def advanced_url_title_match(url, title):
        cleaned_url = extract_url(url)
        cleaned_title = clean_text(title)
        
        if not cleaned_url or not cleaned_title:
            return np.nan
       
        strategies = [
            (lambda u, t: 100 if u in t else 0, 0.4),
            
            (lambda u, t: difflib.SequenceMatcher(None, u, t).ratio() * 100, 0.3),
            
            (lambda u, t: len(set(u) & set(t)) / len(set(u+t)) * 100 if u and t else 0, 0.2),
            
            (lambda u, t: 50 if any(char in t for char in u) else 0, 0.1)
        ]
       
        total_score = 0
        for strategy, weight in strategies:
            try:
                score = strategy(cleaned_url, cleaned_title)
                total_score += score * weight
            except Exception as e:
                print(f"Error in strategy: {e}")
       
        return min(max(total_score, 0), 100)

    def handle_url_title_match_score(df):
        df_copy = df.copy()
        
        mask_missing = df_copy['URLTitleMatchScore'].isna()
    
        def safe_match_calculation(row):
            try:
                score = advanced_url_title_match(row['URL'], row['Title'])
                return score if not np.isnan(score) else 0
            except Exception as e:
                print(f"Error calculating match: {e}")
                return 0
        
        df_copy.loc[mask_missing, 'URLTitleMatchScore'] = df_copy.loc[mask_missing].apply(
            safe_match_calculation, 
            axis=1
        )

        return df_copy

    df_processed = handle_url_title_match_score(df)
    display(df_processed[['URL', 'Title', 'URLTitleMatchScore']])
    
    return df_processed

In [30]:
train_set[['Domain', 'Title', 'DomainTitleMatchScore']].head(20)

,Domain,Title,DomainTitleMatchScore
0,www.northcm.ac.th,norhcmac,NaN
1,http://uqr.to,NaN,NaN
2,www.woolworthsrewards.com.au,woolworthsrewrdscom,0.0
3,NaN,NaN,100.0
4,www.nyprowrestling.com,nyprowrestling,100.0
5,https://www.free-marine.com,NaN,100.0
6,www.ridemcts.com,NaN,100.0
7,https://www.epner.com,epner,100.0
8,www.az511.com,az511,NaN
9,www.screenbeam.com,screenbeam,100.0


In [31]:
# Fill missing isHTTPS according to URL
train_set = fill_missing_isHTTPS(train_set)

missing_data_summary(train_set[['IsHTTPS']])

,URL,IsHTTPS
0,https://www.northcm.ac.th,1
1,http://uqr.to/1il1z,0
2,https://www.woolworthsrewards.com.au,1
3,NaN,0
4,https://www.nyprowrestling.com,1
...,...,...
140399,www.slavevoyages.org,0
140400,https://www.greenmountainenergy.com,1
140401,https://www.leadcastingcall.com,1
140402,https://www.fedarb.com,1


,Feature,Missing Values,Percentage (%)
IsHTTPS,IsHTTPS,0,0.0


In [32]:
# Fill missing HasTitle according to Title
train_set = fill_missing_hasTitle(train_set)

missing_data_summary(train_set[['HasTitle']])

,Title,HasTitle
0,norhcmac,1.0
1,NaN,1.0
2,woolworthsrewrdscom,1.0
3,NaN,0.0
4,nyprowrestling,1.0
...,...,...
140399,slavevoyages,1.0
140400,greenmountainenergy,1.0
140401,leadcastingcall,1.0
140402,NaN,0.0


,Feature,Missing Values,Percentage (%)
HasTitle,HasTitle,0,0.0


In [33]:
# Fill missing Title with Domain 
train_set = fill_missing_title(train_set)

missing_data_summary(train_set[['Title']])

,Domain,Title,DomainTitleMatchScore
0,www.northcm.ac.th,norhcmac,NaN
1,http://uqr.to,NaN,NaN
2,www.woolworthsrewards.com.au,woolworthsrewrdscom,0.0
3,NaN,NaN,100.0
4,www.nyprowrestling.com,nyprowrestling,100.0
...,...,...,...
140399,www.slavevoyages.org,slavevoyages,NaN
140400,www.greenmountainenergy.com,greenmountainenergy,NaN
140401,www.leadcastingcall.com,leadcastingcall,100.0
140402,https://www.fedarb.com,fedarb,100.0


,Feature,Missing Values,Percentage (%)
Title,Title,36420,25.939432


In [34]:
# Fill missing DomainTitleMatchScore according to Domain and Title 
train_set = fill_missing_domaintitle(train_set)

missing_data_summary(train_set[['DomainTitleMatchScore']])

,Domain,Title,DomainTitleMatchScore
0,www.northcm.ac.th,norhcmac,38.200000
1,http://uqr.to,NaN,0.000000
2,www.woolworthsrewards.com.au,woolworthsrewrdscom,0.000000
3,NaN,NaN,100.000000
4,www.nyprowrestling.com,nyprowrestling,100.000000
...,...,...,...
140399,www.slavevoyages.org,slavevoyages,46.714286
140400,www.greenmountainenergy.com,greenmountainenergy,49.065934
140401,www.leadcastingcall.com,leadcastingcall,100.000000
140402,https://www.fedarb.com,fedarb,100.000000


,Feature,Missing Values,Percentage (%)
DomainTitleMatchScore,DomainTitleMatchScore,0,0.0


In [35]:
# Fill missing URLTitleMatchScore according to Domain and Title 
train_set = fill_missing_urltitle(train_set)

missing_data_summary(train_set[['URLTitleMatchScore']])

,URL,Title,URLTitleMatchScore
0,https://www.northcm.ac.th,norhcmac,43.412698
1,http://uqr.to/1il1z,NaN,100.000000
2,https://www.woolworthsrewards.com.au,woolworthsrewrdscom,0.000000
3,NaN,NaN,100.000000
4,https://www.nyprowrestling.com,nyprowrestling,100.000000
...,...,...,...
140399,www.slavevoyages.org,slavevoyages,46.714286
140400,https://www.greenmountainenergy.com,greenmountainenergy,100.000000
140401,https://www.leadcastingcall.com,leadcastingcall,100.000000
140402,https://www.fedarb.com,fedarb,95.000000


,Feature,Missing Values,Percentage (%)
URLTitleMatchScore,URLTitleMatchScore,0,0.0


In [36]:
missing_data_summary(train_set)

,Feature,Missing Values,Percentage (%)
HasObfuscation,HasObfuscation,140339,99.953705
LineOfCode,LineOfCode,69153,49.252870
LargestLineLength,LargestLineLength,67928,48.380388
LetterRatioInURL,LetterRatioInURL,65746,46.826301
ObfuscationRatio,ObfuscationRatio,64598,46.008661
SpacialCharRatioInURL,SpacialCharRatioInURL,62834,44.752286
DegitRatioInURL,DegitRatioInURL,53508,38.110025
TLDLegitimateProb,TLDLegitimateProb,52873,37.657759
URLCharProb,URLCharProb,52071,37.086550
CharContinuationRate,CharContinuationRate,48042,34.216974


### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [37]:
import pandas as pd

# Create a copy of the original DataFrame for comparison
original_train_set = train_set.copy()

# Define custom bounds for specific columns
custom_bounds = {
    'URLLength': (0, None),
    'DomainLength': (0, None),
    'IsDomainIP': (0, 1),
    'CharContinuationRate': (0, None),
    'TLDLegitimateProb': (0, 1),
    'URLCharProb': (0, 1),
    'TLDLength': (0, None),
    'NoOfSubDomain': (0, None),
    'HasObfuscation': (0, 1),
    'NoOfObfuscatedChar': (0, None),
    'ObfuscationRatio': (0, 1),
    'NoOfLettersInURL': (0, None),
    'LetterRatioInURL': (0, 1),
    'NoOfDegitsInURL': (0, None),
    'DegitRatioInURL': (0, 1),
    'NoOfEqualsInURL': (0, None),
    'NoOfQMarkInURL': (0, None),
    'NoOfAmpersandInURL': (0, None),
    'NoOfOtherSpecialCharsInURL': (0, None),
    'SpacialCharRatioInURL': (0, 1),
    'IsHTTPS': (0, 1),
    'LineOfCode': (0, None),
    'LargestLineLength': (0, None),
    'HasTitle': (0, 1),
    'DomainTitleMatchScore': (0, 100),
    'URLTitleMatchScore': (0, 100),
    'HasFavicon': (0, 1),
    'Robots': (0, 1),
    'IsResponsive': (0, 1),
    'NoOfURLRedirect': (0, 1),
    'NoOfSelfRedirect': (0, 1),
    'HasDescription': (0, 1),
    'NoOfPopup': (0, None),
    'NoOfiFrame': (0, None),
    'HasExternalFormSubmit': (0, 1),
    'HasSocialNet': (0, 1),
    'HasSubmitButton': (0, 1),
    'HasHiddenFields': (0, 1),
    'HasPasswordField': (0, 1),
    'Bank': (0, 1),
    'Pay': (0, 1),
    'Crypto': (0, 1),
    'HasCopyrightInfo': (0, 1),
    'NoOfImage': (0, None),
    'NoOfCSS': (0, None),
    'NoOfJS': (0, None),
    'NoOfSelfRef': (0, None),
    'NoOfEmptyRef': (0, None),
    'NoOfExternalRef': (0, None),
    'Label': (0, 1)
}

# Extract numerical columns from the DataFrame
numeric_columns = train_set.select_dtypes(include=['int64', 'float64']).columns.tolist()
outlier_info = {}

for col in numeric_columns:
    if col == 'id': continue  # Skip ID column if it exists

    # Check if custom bounds are defined for the column
    if col in custom_bounds:
        lower_bound, upper_bound = custom_bounds[col]

        # Adjust upper bound based on Q3 if applicable
        if upper_bound is None:
            upper_bound = train_set[col].quantile(0.75)

        if lower_bound is not None:
            train_set.loc[train_set[col] < lower_bound, col] = lower_bound
        if upper_bound is not None:
            train_set.loc[train_set[col] > upper_bound, col] = upper_bound

        # For binary columns (0 or 1), ensure values are strictly 0 or 1
        if set(train_set[col].dropna().unique()).issubset({0, 1}):
            if col == 'Label':
                # Set outliers to NaN for the 'Label' column
                train_set.loc[(train_set[col] < lower_bound) | (train_set[col] > upper_bound), col] = pd.NA
            else:
                train_set[col] = train_set[col].round().clip(0, 1)

        # For columns with values between 0 and 1, clip to [0, 1]
        if lower_bound == 0 and upper_bound == 1:
            train_set[col] = train_set[col].clip(lower=0, upper=1)

        # Skip IQR logic for these columns
        continue

    # Default to IQR method for columns without custom bounds
    Q1, Q3 = train_set[col].quantile(0.25), train_set[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound, upper_bound = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
    outliers = train_set[(train_set[col] < lower_bound) | (train_set[col] > upper_bound)]

    train_set.loc[train_set[col] < lower_bound, col] = lower_bound
    train_set.loc[train_set[col] > upper_bound, col] = upper_bound

    outlier_info[col] = {
        'Jumlah Outlier': len(outliers),
        'Lower Bound': lower_bound,
        'Upper Bound': upper_bound,
        'Outliers': outliers.index.tolist()[:10] if not outliers.empty else '-',
    }

# Convert outlier info to a DataFrame
outlier_df = pd.DataFrame(outlier_info).T
outlier_df['Persentase Outlier (%)'] = (outlier_df['Jumlah Outlier'] / train_set.shape[0]) * 100
outlier_df['Column'] = outlier_df.index

# Reorganize and format columns
outlier_df = outlier_df[['Column', 'Jumlah Outlier', 'Lower Bound', 'Upper Bound', 'Outliers', 'Persentase Outlier (%)']]
outlier_df['Lower Bound'] = outlier_df['Lower Bound'].map(lambda x: f'{x:.2f}' if x is not None else '-')
outlier_df['Upper Bound'] = outlier_df['Upper Bound'].map(lambda x: f'{x:.2f}' if x is not None else '-')
outlier_df['Persentase Outlier (%)'] = outlier_df['Persentase Outlier (%)'].map(lambda x: f'{x:.2f}')

# Display the DataFrame with custom formatting
print("Before Modification:")
print(original_train_set.describe())

print("\nAfter Modification:")
print(train_set.describe())

outlier_df.style.set_properties(**{'text-align': 'center'})\
           .set_table_styles([{'selector': 'th', 'props': [('text-align', 'center')]}])\
           .hide(axis="index")


Before Modification:
                  id      URLLength   DomainLength     IsDomainIP  \
count  140404.000000  130979.000000  133181.000000  133884.000000   
mean   117682.632746      27.760297      20.573475       0.000433   
std     68122.005080      19.330220       6.381912       0.020809   
min         1.000000       4.000000       4.000000       0.000000   
25%     58689.500000      23.000000      16.000000       0.000000   
50%    117421.500000      26.000000      20.000000       0.000000   
75%    176724.250000      30.000000      24.000000       0.000000   
max    235795.000000    4054.000000     110.000000       1.000000   

       CharContinuationRate  TLDLegitimateProb   URLCharProb      TLDLength  \
count          92362.000000       87531.000000  88333.000000  133385.000000   
mean               0.918007           0.277482      0.059286       2.729970   
std                0.159142           0.248115      0.008063       0.531896   
min                0.000000           0.0

Column,Jumlah Outlier,Lower Bound,Upper Bound,Outliers,Persentase Outlier (%)
label,10554,1.00,1.00,"[1, 28, 34, 65, 76, 90, 105, 114, 180, 183]",7.52


### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [38]:
# Write your code here

### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

In [39]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.feature_selection import VarianceThreshold
from fancyimpute import KNN, IterativeImputer  # FancyImpute for advanced missing value handling


# 1. Handle Missing Values with FancyImpute
# Convert categorical data to numerical representation (temporarily)
categorical_cols = train_set.select_dtypes(include=['object']).columns
train_set[categorical_cols] = train_set[categorical_cols].apply(lambda x: x.astype('category').cat.codes)

# Use FancyImpute for Missing Value Imputation
# Impute Numerical Columns with KNN
numerical_cols = train_set.select_dtypes(include=['int64', 'float64']).columns
knn_imputer = KNN(k=5)  # KNN imputation with 5 nearest neighbors
train_set[numerical_cols] = knn_imputer.fit_transform(train_set[numerical_cols])

# If you prefer Iterative Imputer (similar to MICE)
# imputer = IterativeImputer(max_iter=10, random_state=0)
# train_set[numerical_cols] = imputer.fit_transform(train_set[numerical_cols])

# Revert categorical columns back to object dtype
train_set[categorical_cols] = train_set[categorical_cols].astype('category')

# 2. Feature Scaling
scaler = MinMaxScaler()  # Use StandardScaler() for normalization
train_set[numerical_cols] = scaler.fit_transform(train_set[numerical_cols])

# 3. Feature Selection - Variance Threshold (remove low-variance features)
selector = VarianceThreshold(threshold=0.01)  # Adjust threshold as needed
train_set = pd.DataFrame(selector.fit_transform(train_set), columns=train_set.columns[selector.get_support()])

# 4. Creating New Features
# Polynomial Features
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
numerical_poly = poly.fit_transform(train_set[numerical_cols])
poly_features = poly.get_feature_names_out(numerical_cols)
train_set = pd.concat([train_set, pd.DataFrame(numerical_poly, columns=poly_features)], axis=1)

# Interaction Features - Example: URLLength and DomainLength interaction
train_set['URL_Domain_Interaction'] = train_set['URLLength'] * train_set['DomainLength']

# Domain-Specific Features
train_set['LegitRate_CharContinuation'] = train_set['TLDLegitimateProb'] * train_set['CharContinuationRate']

# 5. Binning/Discretization
bins = [0, 10, 20, 30, 40]
labels = ['Very Short', 'Short', 'Medium', 'Long']
train_set['URLLength_binned'] = pd.cut(train_set['URLLength'], bins=bins, labels=labels)

# 6. Drop Irrelevant or Redundant Columns
if 'id' in train_set.columns:
    train_set.drop(columns=['id'], inplace=True)

correlation_matrix = train_set.corr().abs()
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
high_corr_features = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.95)]
train_set.drop(columns=high_corr_features, inplace=True)

# 7. Encoding Categorical Variables
train_set = pd.get_dummies(train_set, drop_first=True)

# Save the transformed dataset

print("Feature Engineering Completed!")


## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

In [40]:
# Write the code

# from sklearn.base import BaseEstimator, TransformerMixin

# class FeatureEncoder(BaseEstimator, TransformerMixin):

#     def fit(self, X, y=None):

#         # Fit the encoder here

#         return self

#     def transform(self, X):
#         X_encoded = X.copy()

#         # Encode the categorical variables here

#         return X_encoded

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [41]:
# Write your code here

### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [42]:
# Write your code here

### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [43]:
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.over_sampling import SMOTE

class CustomSMOTE(BaseEstimator, TransformerMixin):
    def __init__(self, sampling_strategy='auto', random_state=42):
        self.smote = SMOTE(sampling_strategy=sampling_strategy, random_state=random_state)

    def fit(self, X, y=None):
        self.smote.fit(X, y)
        return self

    def transform(self, X, y=None):
        X_resampled, y_resampled = self.smote.fit_resample(X, y)
        return X_resampled, y_resampled

# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [44]:
# from sklearn.pipeline import Pipeline

# # Note: You can add or delete preprocessing components from this pipeline

# pipe = Pipeline([("imputer", FeatureImputer()),
#                  ("featurecreator", FeatureCreator()),
#                  ("scaler", FeatureScaler()),
#                  ("encoder", FeatureEncoder())])

# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

In [45]:
# # Your code should work up until this point
# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

or create your own here

In [46]:
# Write your code here

# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `label`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

## A. KNN

In [ ]:
# Assuming train_set and val_set are pre-split DataFrames
# Step 1: Handle Missing Values
numerical_cols = train_set.select_dtypes(include=['float64', 'int64']).columns.drop(['id', 'label'])

# Fill missing values with train_set means for both sets
train_set[numerical_cols] = train_set[numerical_cols].fillna(train_set[numerical_cols].mean())
val_set[numerical_cols] = val_set[numerical_cols].fillna(train_set[numerical_cols].mean())

# Optional: Drop object columns or encode them
columns_to_drop = ['URL', 'Domain', 'TLD', 'Title']
train_set = train_set.drop(columns=columns_to_drop, errors='ignore')
val_set = val_set.drop(columns=columns_to_drop, errors='ignore')

# Align the validation set columns with training set columns
val_set = val_set[train_set.columns]

# Step 2: Standardize Features
scaler = StandardScaler()
X_train = scaler.fit_transform(train_set.drop(columns=['label']))
y_train = train_set['label']
X_val = scaler.transform(val_set.drop(columns=['label']))
y_val = val_set['label']

# Define distance metrics to evaluate
distance_metrics = ['euclidean', 'manhattan', 'minkowski', 'chebyshev']

# Loop through metrics
for metric in distance_metrics:
    print(f"Testing KNN with metric: {metric}")
    
    # Initialize and train KNN
    knn = KNeighborsClassifier(n_neighbors=5, metric=metric)
    knn.fit(X_train, y_train)
    
    # Validate the model
    y_val_pred = knn.predict(X_val)
    print(f"Validation Metrics for {metric}:")
    print(classification_report(y_val, y_val_pred))
    val_f1_score = f1_score(y_val, y_val_pred, average='macro')
    print(f"Validation F1 Score (Macro) for {metric}: {val_f1_score:.4f}\n")

# Optional: Save the best-performing model
best_metric = 'euclidean'  # Replace with your chosen metric after evaluation
best_knn = KNeighborsClassifier(n_neighbors=5, metric=best_metric)
best_knn.fit(X_train, y_train)
joblib.dump(best_knn, f"knn_model_{best_metric}.pkl")
print(f"Best model saved as 'knn_model_{best_metric}.pkl'")


Testing KNN with metric: euclidean
Validation Metrics for euclidean:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2111
           1       0.92      1.00      0.96     25970

    accuracy                           0.92     28081
   macro avg       0.46      0.50      0.48     28081
weighted avg       0.86      0.92      0.89     28081

Validation F1 Score (Macro) for euclidean: 0.4805

Testing KNN with metric: manhattan


C:\Users\FAVIAN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\FAVIAN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\FAVIAN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classific

Validation Metrics for manhattan:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2111
           1       0.92      1.00      0.96     25970

    accuracy                           0.92     28081
   macro avg       0.46      0.50      0.48     28081
weighted avg       0.86      0.92      0.89     28081

Validation F1 Score (Macro) for manhattan: 0.4805

Testing KNN with metric: minkowski


C:\Users\FAVIAN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\FAVIAN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\FAVIAN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classific

Validation Metrics for minkowski:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2111
           1       0.92      1.00      0.96     25970

    accuracy                           0.92     28081
   macro avg       0.46      0.50      0.48     28081
weighted avg       0.86      0.92      0.89     28081

Validation F1 Score (Macro) for minkowski: 0.4805

Testing KNN with metric: chebyshev


C:\Users\FAVIAN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\FAVIAN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\FAVIAN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classific

Validation Metrics for chebyshev:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2111
           1       0.92      1.00      0.96     25970

    accuracy                           0.92     28081
   macro avg       0.46      0.50      0.48     28081
weighted avg       0.86      0.92      0.89     28081

Validation F1 Score (Macro) for chebyshev: 0.4805



C:\Users\FAVIAN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\FAVIAN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\FAVIAN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classific

Best model saved as 'knn_model_euclidean.pkl'


## B. Naive Bayes

In [374]:
# Type your code here

## C. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

In [375]:
# Type your code here

## D. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `label` column.

Note: Adjust step 1 and 2 to your implementation of the preprocessing step if you don't use pipeline API from `sklearn`.

In [376]:
# Type your code here

# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

`Provide your analysis here`